In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# gdrive_home_dir = '/content/drive/My Drive/'


# !ls '{gdrive_home_dir}'







In [ ]:
# !pip install flickrapi

# import flickrapi

In [ ]:
# Photo Source URL description: https://www.flickr.com/services/api/misc.urls.html

# Find the id of a group by group name: https://www.flickr.com/services/api/explore/flickr.urls.lookupGroup
# Get photos from a group: https://www.flickr.com/services/api/explore/flickr.groups.pools.getPhotos

# flickr photo search api: https://www.flickr.com/services/api/explore/flickr.photos.search

# Flickr photo licenses explained: https://www.flickr.com/creativecommons/
# Flick photo licenses: https://www.flickr.com/services/api/explore/flickr.photos.licenses.getInfo

# sample download scripts: https://gist.github.com/zmwangx/b1c16b197b5416143c7a
#                          https://www.programcreek.com/python/example/6468/flickrapi.FlickrAPI
#% reset

In [3]:
'''
https://stackoverflow.com/questions/1994037/flickr-api-returning-duplicate-photos 

It is possible to retrieve more than 4000 images from flickr; your query has to be paginated by (for example) temporal range 
such that the total number of images from that query is not more than 4000. You can also use other parameters such as bounding 
box to limit the total number of images in the response.

For example, if you are searching with the tag 'dogs', this is what you can do ( binary search over time range):

Specify a minimum date and a maximum date in the request url, such as Jan 1st, 1990 and Jan 1st 2015.
Inspect the total number of images in the response. If it is more than 4000, then divide the temporal range into two and 
work on the first half until you get less than 4000 images from the query. Once you get that, request all the pages from 
that time range, and move on to the next interval and do the same until (a) Number of required images is met (b) searched
all over the initial time interval.

'''

#%reload_ext downloader_wrapper

from flickr_dloader_utils import *
import time
import datetime


# Recommended way for creating a multi-process program in python
if __name__ == '__main__':

    
    # number of images to download
    num_images_to_download = 12000 

    # Specify whether to download from a group or by tags
    config = 'tags' # possible values: 'group' or 'tags'

    # base download directory
    #download_dir = f'{gdrive_home_dir}/dataset/test_dataset'
    download_dir = f'E:\\datasets\\tiger_vs_bear\\tiger'

    # name of the group (if downloading from a group, ignored otherwise)
    group_name = 'portrait-gallery'

    # tags to be used to searching (if downloading using search tags, ignored otherwise)
    tags = 'tiger'

    # to try time intervals 
    SECONDS_IN_A_YEAR = 31536000    
    
    CURRENT_TIME = int(time.time())
    
    # maximum number of times to retry if an image info page results in error
    MAX_RETRY_COUNT = 3

    # maximum number of images to assign to an image downloader process
    MAX_LOAD_PER_IMG_DLOADER = 200 
    
    # maximum number of images information to be contained in a query result page.
    # NOTE: Flickr sends the same set of image information if used a number higher than 100.
    MAX_IMG_PER_PAGE = 100    
    
    # hack since api downloads 100 less than the specified number of images
    num_images_to_download +=100 
    
    # finds the number of pages needed to be downloaded
    #num_pages = int(math.ceil(num_images_to_download/MAX_IMG_PER_PAGE))

    #print(f'Number of pages needed for {num_images_to_download} images: {num_pages}')
    
    # settings for downloading images from a group or by search keywords/tags
    configs = { 'group': {
                            'search_criteria' : group_name,
                            'download_dir' : download_dir,    
                            #'image_info_save_file' : 'query_results_bygroup__' + group_name,
                            'url_downloader' : download_flickr_img_by_group, # image info downloader function for groups
                        },
               'tags': {
                            'search_criteria' : tags,
                            'download_dir' : download_dir,    
                            #'image_info_save_file' : 'query_results_bytags__' + ".".join(tags.split(",")),
                            'url_downloader' : download_flickr_img_url_by_tag, # image info downloader function for tags
                        }
              }

    retry_count = 0
    is_page_error = True
    while retry_count < MAX_RETRY_COUNT and is_page_error:  
        try:  
            # execute the query for finding out the number of pages available 
            result = configs[config]['url_downloader'](configs[config]['search_criteria'])
            is_page_error = False 
        except Exception as e:
            # enforced delay not to overwhelm the Flickr API  
            retry_count += 1
            print('Retry attempt: ', retry_count)
            time.sleep(2)    
    
   
    # Number of pages in the result    
    pagecount = result['photos']['pages']
    imgCount = int(result['photos']['total'])
    
    print(f"Total available images: {imgCount}")
    print(f'Total available pages: {pagecount}')

    results = []
    search_time_intervals = []

    # split the date range so that each query results in less than total 4000 images; otherwise flickr returns duplicates
    if(imgCount > 4000):
        print('More than 4k images in query; segmenting the query')
        photo_uploaddates = [photo['dateupload'] for photo in result['photos']['photo'] ]
        start_upload_date = int(photo_uploaddates[0])
        #end_upload_date = start_upload_date + SECONDS_IN_A_YEAR
        
        #print(first_photo_upload_date)                  
        #print(result)
        
        #unique_query_photos = { photo['id']:photo for result in results for photo in result['photos']['photo']}
        #unique_query_photos = {}
        total_img_count = 0
        
        while( total_img_count < num_images_to_download and start_upload_date < CURRENT_TIME ):
            interval_length = (CURRENT_TIME - start_upload_date)//2  #10*SECONDS_IN_A_YEAR
            end_upload_date = start_upload_date + interval_length
            
            start_time_h = datetime.datetime.utcfromtimestamp(start_upload_date).strftime('%Y-%m-%d %H:%M:%SZ')
            end_time_h = datetime.datetime.utcfromtimestamp(end_upload_date).strftime('%Y-%m-%d %H:%M:%SZ')
            
            
            retry_count = 0
            is_page_error = True
            while retry_count < MAX_RETRY_COUNT and is_page_error:  
                try:  
                    # execute the query for finding out the number of pages available 
                    result = configs[config]['url_downloader'](configs[config]['search_criteria'], pagenum = None, 
                                                                                           show_log = False, 
                                start_date = start_upload_date, end_date = end_upload_date)
            
                    is_page_error = False 
                except Exception as e:
                    # enforced delay not to overwhelm the Flickr API  
                    retry_count += 1
                    print('Retry attempt: ', retry_count)
                    time.sleep(1)    
            
            time.sleep(0.5)
            imgCount = int(result['photos']['total'])
            print("image count: ", imgCount, " (between", end="") 
            print("start: ", start_time_h, " end:", end_time_h, ")" )
            
            while(imgCount > 4000 and interval_length > 1):
                
                interval_length //= 2
                end_upload_date = start_upload_date + interval_length
                
                start_time_h = datetime.datetime.utcfromtimestamp(start_upload_date).strftime('%Y-%m-%d %H:%M:%SZ')
                end_time_h = datetime.datetime.utcfromtimestamp(end_upload_date).strftime('%Y-%m-%d %H:%M:%SZ')
                    
                retry_count = 0
                is_page_error = True
                while retry_count < MAX_RETRY_COUNT and is_page_error:  
                    try:  
                        # execute the query for finding out the number of pages available 
                        result = configs[config]['url_downloader'](configs[config]['search_criteria'], pagenum = None, 
                                                                                               show_log = False, 
                                    start_date = start_upload_date, end_date = end_upload_date)

                        is_page_error = False 
                    except Exception as e:
                        # enforced delay not to overwhelm the Flickr API  
                        retry_count += 1
                        print('Retry attempt: ', retry_count)
                        time.sleep(2)                
                
                time.sleep(0.5)
                imgCount = int(result['photos']['total'])
                print("Search... image count: ", imgCount, " (between", end="") 
                print("start: ", start_time_h, " end:", end_time_h, ")" )
            
            # found query with less than 4k images, so add the start and end times to the list 
            search_time_intervals.append( (start_upload_date, end_upload_date ) )
            total_img_count += imgCount
            
            # update start time for the next query
            start_upload_date = end_upload_date + 1
            
            print("Total image count is: ", total_img_count)
            print()
            
    else:
        results.append(result)

        
    print(search_time_intervals)
    
    query_photos = {}
            
    for start_upload_date, end_upload_date in search_time_intervals:
        retry_count = 0
        is_page_error = True
        while retry_count < MAX_RETRY_COUNT and is_page_error:  
            try:  
                # execute the query for finding out the number of pages available 
                # execute the query for finding out the number of pages available 
                result = configs[config]['url_downloader'](configs[config]['search_criteria'], pagenum = None, 
                                                                                           show_log = False, 
                                start_date = start_upload_date, end_date = end_upload_date)
                is_page_error = False 
            except Exception as e:
                # enforced delay not to overwhelm the Flickr API  
                retry_count += 1
                print('Retry attempt: ', retry_count)
                time.sleep(2)    


            # Number of pages in the result    
            pagecount = result['photos']['pages']
        
            page_idx = 0

            
            has_stalled = False

            image_counts = []  
            # download image info (can't be done parallely since can't instantiate more than one Flickr api instances with 
            # the same api key: possibly can be done by using one key per instance)          
            while len(query_photos.items()) < num_images_to_download and page_idx < pagecount and not has_stalled:

                page_idx += 1

                is_page_error = True
                
                retry_count = 0

                while retry_count < MAX_RETRY_COUNT and is_page_error:  
                    try:  
                        # download image info as a dictionary and append to a list  
                        result = configs[config]['url_downloader'](configs[config]['search_criteria'], page_idx, 
                                                                           show_log = True, 
                                start_date = start_upload_date, end_date = end_upload_date)
                        
                        is_page_error = False 
                    except Exception as e:
                        # enforced delay not to overwhelm the Flickr API  
                        retry_count += 1
                        print('Retry attempt: ', retry_count)
                        time.sleep(2)

                results.append(result)
                # enforced delay not to overwhelm the Flickr API  
                time.sleep(0.5)

                # construct a dictionary of returned photos to eliminate ducplicate images (based on image id)      
                query_photos_this_page = { photo['id']:photo for photo in result['photos']['photo']}

                query_photos.update(query_photos_this_page)

                # compute the number of image information obtained       
                actual_total_image_count = len(query_photos.items()) # Flickr returns 100 less images for some reason

                image_counts.append(actual_total_image_count)

                print(f'Obtained total {len(query_photos_this_page.items())} image information from {page_idx} pages, total obtained:{actual_total_image_count}')

                if(len(image_counts) > 5 and image_counts[-1] < image_counts[-5] + 20):
                    has_stalled = True
    
    print(f'Attempting to download total {actual_total_image_count} images')          

          
    ## Download the images from the downloaded image information           
          
    # update num_process to reflect actual image count
    num_processes = int(math.ceil(actual_total_image_count/MAX_LOAD_PER_IMG_DLOADER))
              
    print(f'Number of image downloaders: {num_processes}')
              
    # arrange the image downloader arguments for multi-process environment          
    args = get_worker_args(num_processes, query_photos, MAX_LOAD_PER_IMG_DLOADER, configs[config]['download_dir'], 
                            'c', None)
    
    # for suppressing process output
    def mute():
        sys.stdout = open(os.devnull, 'w')   
              
    start = time.time()
              
    with Pool(num_processes) as p:
        p.map(downloader_wrapper, args);
    
    end = time.time()
    
    print(f'Elapsed time {end-start} seconds')
              

Total available images: 41411
Total available pages: 415
More than 4k images in query; segmenting the query
image count:  20522  (betweenstart:  2004-12-28 04:03:41Z  end: 2012-02-17 09:24:52Z )
Search... image count:  7627  (betweenstart:  2004-12-28 04:03:41Z  end: 2008-07-23 18:44:16Z )
Search... image count:  2376  (betweenstart:  2004-12-28 04:03:41Z  end: 2006-10-10 23:23:58Z )
Total image count is:  2376

image count:  20852  (betweenstart:  2006-10-10 23:23:59Z  end: 2013-01-08 07:05:01Z )
Search... image count:  10659  (betweenstart:  2006-10-10 23:23:59Z  end: 2009-11-24 15:14:30Z )
Search... image count:  4203  (betweenstart:  2006-10-10 23:23:59Z  end: 2008-05-03 07:19:14Z )
Search... image count:  1977  (betweenstart:  2006-10-10 23:23:59Z  end: 2007-07-23 03:21:36Z )
Total image count is:  4353

image count:  19932  (betweenstart:  2007-07-23 03:21:37Z  end: 2013-05-30 21:03:50Z )
Search... image count:  10979  (betweenstart:  2007-07-23 03:21:37Z  end: 2010-06-26 12:12:4

do_request: Status code 502 received, content:
    <!DOCTYPE html>
<html lang="en-us"><head>
<meta http-equiv="content-type" content="text/html; charset=UTF-8">
    <meta charset="utf-8">
    <title>Yahoo</title>
    <meta name="viewport" content="width=device-width,initial-scale=1,minimal-ui">
    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
    <style>
html {
    height: 100%;
}
body {
    background: #fafafc url(https://s.yimg.com/nn/img/sad-panda-201402200631.png) 50% 50%;
    background-size: cover;
    height: 100%;
    text-align: center;
    font: 300 18px "helvetica neue", helvetica, verdana, tahoma, arial, sans-serif;
}
table {
    height: 100%;
    width: 100%;
    table-layout: fixed;
    border-collapse: collapse;
    border-spacing: 0;
    border: none;
}
h1 {
    font-size: 42px;
    font-weight: 400;
    color: #400090;
}
p {
    color: #1A1A1A;
}
#message-1 {
    font-weight: bold;
    margin: 0;
}
#message-2 {
    display: inline-block;
    *display:

Retry attempt:  1
Search... image count:  2461  (betweenstart:  2009-08-14 02:01:22Z  end: 2010-03-22 08:49:09Z )
Total image count is:  14313

[(1104206621, 1160522638), (1160522639, 1185160896), (1185160897, 1208259263), (1208259264, 1229913982), (1229913983, 1250215281), (1250215282, 1269247749)]
Obtained total 99 image information from 1 pages, total obtained:99
Obtained total 100 image information from 2 pages, total obtained:199
Obtained total 99 image information from 3 pages, total obtained:298
Obtained total 100 image information from 4 pages, total obtained:398
Obtained total 100 image information from 5 pages, total obtained:498
Obtained total 100 image information from 6 pages, total obtained:598
Obtained total 100 image information from 7 pages, total obtained:698
Obtained total 100 image information from 8 pages, total obtained:798
Obtained total 100 image information from 9 pages, total obtained:898
Obtained total 100 image information from 10 pages, total obtained:998
O

Obtained total 100 image information from 21 pages, total obtained:8566
Obtained total 100 image information from 22 pages, total obtained:8666
Obtained total 100 image information from 23 pages, total obtained:8766
Obtained total 100 image information from 24 pages, total obtained:8866
Obtained total 100 image information from 25 pages, total obtained:8966
Obtained total 100 image information from 26 pages, total obtained:9066
Obtained total 100 image information from 27 pages, total obtained:9166
Obtained total 76 image information from 28 pages, total obtained:9242
Obtained total 98 image information from 1 pages, total obtained:9340
Obtained total 100 image information from 2 pages, total obtained:9440
Obtained total 100 image information from 3 pages, total obtained:9540
Obtained total 100 image information from 4 pages, total obtained:9640
Obtained total 99 image information from 5 pages, total obtained:9739
Obtained total 99 image information from 6 pages, total obtained:9838
Ob